In [ ]:
!conda install -y -c astra-toolbox/label/dev astra-toolbox

In [ ]:
import os
import glob
import numpy as np
import scipy as sp
import nibabel as nib
from matplotlib import pyplot as plt
%matplotlib inline

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

normalize = lambda img : (img - np.amin(img)) / (np.amax(img) - np.amin(img))

import astra
import numpy as np
import pydicom
import time
import os

In [ ]:
import math
import time
from random import randint
from skimage.morphology.selem import ball, cube, diamond
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean

def normalize(img):
    # USED FOR NORMALIZATION
    img -= img.min()
    img /= img.max()
    return img

def normalize_vector(v):
    return v / np.sqrt(np.sum((v ** 2)))

def point_to_int(origin):
    return [int(round(origin[0])),  int(round(origin[1])), int(round(origin[2]))]

def check_inside(origin, shape):
    r_origin = point_to_int(origin)
    if r_origin[0] < 0 or r_origin[0] >= shape[0]:
        return False
    if r_origin[1] < 0 or r_origin[1] >= shape[1]:
        return False
    if r_origin[2] < 0 or r_origin[2] >= shape[2]:
        return False
    return True

def fill_mask(origin, mask, width_needle):
    mask[origin[0]:origin[0] + width_needle,
         origin[1]:origin[1] + width_needle,
         origin[2]:origin[2] + width_needle] = True

def generate_3d_artifact(img, max_nb_artefacts=4, min_len_needle=200, 
                         max_len_needle=500, width_needle=10, size_artefacts=10):
    len_artefacts = np.random.randint(min_len_needle, max_len_needle)
    line_nb = np.random.randint(1, max_nb_artefacts)
    mask = np.zeros(img.shape, dtype=bool)
    for _ in range(line_nb):
        origin = np.array([randint(0, img.shape[x]) for x in range(len(img.shape))], dtype=np.float32)
        direction = np.array([randint(-10,10), randint(-10,10), randint(-10,10)], dtype=np.float32)
        direction = normalize_vector(direction)
        l = 0
        while check_inside(origin + width_needle, img.shape) or check_inside(origin, img.shape) or  l < len_artefacts:
            r_origin = [int(round(x)) for x in origin]
            fill_mask(r_origin, mask, width_needle)
            origin += direction
            l += 1
    
    coil_nb = np.random.randint(1, max_nb_artefacts)
    mult_artefacts = np.array([ball, cube, diamond])
    radius = size_artefacts // 2
    artefact_obj = np.random.choice(mult_artefacts)(radius)
    for _ in range(coil_nb):
        origin = np.array([randint(0, img.shape[x] - size_artefacts) for x in range(len(img.shape))], dtype=np.int)
        mask[origin[0]:origin[0] + size_artefacts, 
             origin[1]:origin[1] + size_artefacts, 
             origin[2]:origin[2] + size_artefacts] = resize(artefact_obj, (size_artefacts, size_artefacts, size_artefacts))
    return mask

def create_3d_image_artifact(img, mask):
    artifacted = img.copy()
    mask = generate_3d_artifact(img, mask)
    artifacted[mask] = 1.0
    return artifacted

def fill_image_with_mask(img, mask):
    artifacted = img.copy()
    artifacted[mask] = np.max(artifacted) * 10.0
    return artifacted

In [ ]:
def load_from_dicom_folder(address_case):
    """
    Load a volume and the voxel sizes from a folder containing the dicom files
    of the reformatted slices.\n

    Parameters
    ----------
    address_case : str
        Full path to the folder
    Returns
    ----------
    vol : numpy.array
        the loaded volume
    l : list of floats
        the voxel size in in mm in x,y,z
    2 : list of floats
        position of the center of first voxel of the volume
    """

    # TODO : Adapt those 3 lines to your case.
    files = os.listdir(address_case)
    files = [f for f in files if f[-4:] == ".dcm"]
    files = sorted(files, key = lambda x: int(x[2:5]))

    assert(
            len(files) > 0,
            f"no CTDC files found in directory {address_case}"
    )

    imgList = []
    for f in files:
        im = pydicom.read_file(address_case+"/"+f)
        imgList.append((im.ImagePositionPatient[2], im))
    imgList = sorted(imgList)

    vol = np.array(
            [imdcm.pixel_array for _, imdcm in imgList],
            dtype = imgList[0][1].pixel_array.dtype
    )
    vol[vol==-2000] = 0

    im0 = imgList[0][1]
    pix_ct = list(map(float, im0.PixelSpacing))
    try:
        slice_ct = abs(im0.SpacingBetweenSlices.real)
    except:
        print("SpacingBetweenSlices not found. use SliceThickness instead")
        slice_ct = abs(im0.SliceThickness.real)
    imageOrigin = im0.ImagePositionPatient
    vox_sizes = pix_ct + [slice_ct]
    print("Volume shape is {}".format(vol.shape))
    print("Voxel size is {}".format(vox_sizes))
    print("Volume origin is {}".format(imageOrigin))
#    ratio_ct=slice_ct/pix_ct
    return vol,vox_sizes, imageOrigin

In [ ]:
from scipy.ndimage import zoom
def make_isotropic(
        vol,vox_sizes,order_of_interpolation=1, vox_size_iso = None
):
    """
    Wrap the scipy.nidimage.zoom interpolation to make ct volumes isotropic in
    the z dimension.
    Although not clearly documented, resize considers the center
    of the first and last voxel of a volume.
    This means that origin of the volume (center of the fist voxel) remains
    unchanged.

    Parameters
    ----------
    vol : numpy.array
        the volume to turn iostropic
    vox_sizes: list of float
        the voxel size in x,y,z
    vox_size_iso (optional):
        final voxel size after resize
        byt default, final isotropic voxel size will be vox_sizes[0]

    Returns
    ----------
    vol_iso : numpy.array
        the interpolated volume
    """

    if vox_size_iso == None:
        ratio_ct = np.array(vox_sizes)[::-1]/vox_sizes[0]
    else:
        ratio_ct = np.array(vox_sizes)[::-1]/vox_size_iso
    return zoom(vol,ratio_ct,order=order_of_interpolation)

In [ ]:
def computeCamToXrayRotWithPCangles(Pdeg, Cdeg, Ldeg=0):
    """
        returns the rotation from Camera CS (ie rotated with P, C) toXray CS.
        Cra and Lao are positive. L positive moves gantry to patient left.
        Cam CS has the same orientation as Xray CS. X is optical axis but
        pointing downwards,
               Y oriented with the lines of detector to the right
               Z oriented with the columns of the detector to patient feet
    """
    sC = np.sin(np.radians(-Cdeg)) #minus because C positive is cra
    cC = np.cos(np.radians(-Cdeg))
    sP = np.sin(np.radians(-Pdeg)) #minus because P positive is lao
    cP = np.cos(np.radians(-Pdeg))
    sL = np.sin(np.radians(Ldeg))
    cL = np.cos(np.radians(Ldeg))
    RP = np.array([[cP, -sP, 0], [sP, cP, 0],[0,0,1]])
    RC = np.array([[cC, 0, sC],[0,1,0], [-sC, 0, cC]])
    RL = np.array([[1, 0, 0],[0,cL,-sL], [0, sL, cL]])
    return np.linalg.multi_dot((RL,RP,RC));

def computeCameramWithPCangles(Pdeg, Cdeg, SOD, SID, pixSize, Ldeg=0):
    """
        transform system paramters into projection matrix
        world is Xray reference frame
           Cra and Lao are positive. L positive moves gantry to patient left.
    """
    camToXray = computeCamToXrayRotWithPCangles(Pdeg, Cdeg, Ldeg)
    FS = SOD*camToXray[:,0]
    kext = np.linalg.inv(camToXray).dot(np.vstack((np.identity(3), -FS )).T)
    kswap = np.zeros((3,3))
    kswap[2,0] = kswap[0,1] = kswap[1,2] = 1
    kint = np.identity(3)
    kint[0,0] = kint[1,1] = SID/pixSize
    kint[2,2] = -1
    return kint.dot(kswap).dot(kext)


def decomposeCameraFSdetector2(tdir, pixSize):
    """
    turn a projection matrix obtained from rvFeldkamp into a camera vector
    expected by astra.\n

    Parameters
    ----------
    tdir : 3x4 numpy.array
        projection matrix
    pixSize : float

    Returns
    ----------
    opticalCenter : 3x1 np.array

    detOrigin : 3x1 np.array

    U : 3x1 np.array
        the vector from detector pixel (0,0) to (0,1)
    V : 3x1 np.array
        the vector from detector pixel (0,0) to (1,0)
    """
    tdir = tdir / np.linalg.norm(tdir[2,:2])
    tdir3 = tdir[:,:3]
    b = -tdir[:,3]
    invtdir3 = np.linalg.inv(tdir3)
    opticalCenter = invtdir3.dot(b);
    normFactor = pixSize/np.mean(np.linalg.norm(invtdir3[:,:2], axis = 0))
    detOrigin = opticalCenter + (invtdir3[:,2]*normFactor)
    U = invtdir3[:,0] * normFactor
    V = invtdir3[:,1] * normFactor
    return(opticalCenter, detOrigin, U, V)

def generate_camera_vector(
        pos,
        vox_sizes_mm,
        shape_input,
        proj_mat_Xray_mm,
        input_img_size,
        output_img_size
):
    """
    transforms the input projection matrices expressed in Xray CS
    into a camera vector expected by astra.\n

    Parameters
    ----------
    pos : list of floats
        Coordinates in voxels (indz,indy,indx) of the point of the volume to
        project to put at the center of the FOV
    vox_sizes_mm : list of floats
        Size of the volume voxels in mm (order is (x,y,z))
    shape_input : tuple of ints
        Size of the volume to project (order is (z,y,x))
    proj_mat_Xray_mm : list of [3,4] arrays
        list of projection matrix
    input_img_size : int
        size in pixel of input image
    output_img_size : int
        size in pixel of output image allows image resize
    Returns
    ----------
    v : list of 12x1 np.array
        list of camera vectors corresponding
    """
    Mdet_center = np.eye(3)
    Mdet_center[:2,2]=-(input_img_size/2)
    Mdet_resize =np.diag([output_img_size/input_img_size]*2+[1])

    Swap_xray_to_ct=np.zeros([4,4])
    Swap_xray_to_ct[3,3] = 1
    Swap_xray_to_ct[2,2] = -1
    Swap_xray_to_ct[0,1] = 1
    Swap_xray_to_ct[1,0] = 1

    Mscale = np.diag(list(vox_sizes_mm)+[1.0])
    # place in t the coordinates of center in voxels - reverse order because
    # pos is returned in z, y, x
    t = -(np.array(pos)[::-1]-np.array(shape_input)[::-1]/2)
    Mtranslation = np.eye(4)
    Mtranslation[0:3,3]=t

    proj_mat_ct = [
            np.linalg.multi_dot((
                Mdet_resize,
                Mdet_center,
                p,
                Swap_xray_to_ct,
                Mscale,
                Mtranslation)) for p in proj_mat_Xray_mm
    ]
    v = np.vstack([
        np.concatenate(decomposeCameraFSdetector2(p, 1.0)) for p in proj_mat_ct
    ])
    return v

In [ ]:
import matplotlib.pyplot as plt
''' interactive display of image series
    seq is a Nframe*nrows*ncolumns array
    capture mouse wheel, keyboqrd up,down,pageup,pagedown to page through
    images/slices of input array
'''
def displaySequence(seq, vmin = None, vmax = None):
    global curr_frame
    curr_frame = 0#current frame
    if vmin == None:
        vmin = seq.min()
    vmin = float(vmin)
    if vmax == None:
        vmax = seq.max()
    vmax = float(vmax)

    def key_event(e):#response to keyboard input
#        print('you pressed', e.key, e.xdata, e.ydata)
        global curr_frame
        #if arrows on the keyboard are pressed, change curr_frame
        if e.key == "up":
            curr_frame = curr_frame + 1
        elif e.key == "down":
            curr_frame = curr_frame - 1
        elif e.key == "pageup":
            curr_frame = curr_frame + 5
        elif e.key == "pagedown":
            curr_frame = curr_frame - 5
        else:
            return
        #handle end of the series
        curr_frame = curr_frame % seq.shape[0]
        #refresh axis
        ax.cla()#clear axis
        ax.imshow(seq[curr_frame], cmap='gray', vmin = vmin, vmax = vmax)
        ax.set_title("frame {0}".format(int(curr_frame)))
        fig.canvas.draw()#refresh

    def on_scroll(e):#response to mouse scroll input
        global curr_frame
        # update the frame with respect to the scroll input
        curr_frame = curr_frame - int(e.step)
        # handle end of series
        curr_frame = curr_frame % seq.shape[0]
        #refresh axis
        ax.cla()  # clear axis
        ax.imshow(seq[curr_frame], cmap='gray', vmin = vmin, vmax = vmax)
        ax.set_title("frame {0}".format(int(curr_frame)))
        fig.canvas.draw()  # refresh

    fig = plt.figure()  # Create figure
    # Connect the keyboard listener function key_event to the figure
    fig.canvas.mpl_connect('key_press_event', key_event)
    # connect the scroll listener function on_scroll to the figure
    fig.canvas.mpl_connect('scroll_event', on_scroll)
    ax = fig.add_subplot(111)  # Add axis to figure
    ax.imshow(seq[curr_frame], cmap='gray', vmin = vmin, vmax = vmax)
    ax.set_title("frame {0}".format(int(curr_frame)))
    plt.show()  # Show figure

In [ ]:
import scipy
from skimage.measure import label
from skimage.filters import threshold_otsu

In [ ]:
def compute_mask(images):
    masks = np.zeros(shape=images.shape, dtype=np.bool)
    for i, image in enumerate(images):
        thresh = threshold_otsu(image)
        bin_img = image > thresh
        filled_image = scipy.ndimage.morphology.binary_fill_holes(bin_img)
        labels = label(filled_image)
        assert( labels.max() != 0 ) # assume at least 1 CC
        final_mask = labels == np.argmax(np.bincount(labels.flat)[1:])+1
        masks[i] = final_mask
    return masks

In [ ]:
import glob

In [ ]:
DATA_PATH = '../input/dataset-ct/dataset_ct/covid19/'
IMAGE_PATH = DATA_PATH

In [ ]:
def generate_3d_dataset_artifact(filename):
    im = nib.load(filename)
    im_arr = im.get_fdata()

    im_arr = im_arr.T

    #Get the image dimension (voxel)
    print("Image dimension:", im_arr.shape)

    #Get the voxel size (mm)
    print("Voxel size:", im.header.get_zooms())

    voxel_size = im.header.get_zooms()
    
    masks = compute_mask(im_arr)
    masks.shape, masks
    
    mask_art = generate_3d_artifact(im_arr, size_artefacts=10)
    
    sx, sy, sz = im_arr.shape
    
    projection_center = (sx//2, sy//2, sz//2)

    t1 = time.time()

    ct_isotropic = make_isotropic(mask_art, voxel_size, order_of_interpolation=1)

    projection_center_iso = [projection_center[0]*voxel_size[2]/voxel_size[0], projection_center[1], projection_center[2]]
    voxel_size_iso = voxel_size[0]

    t2 = time.time()

    print("Shape of interpolated CT :", ct_isotropic.shape)
    print(f"Time for interpolation : {t2-t1:.3f}s")
    LaoRaolist, CraCaulist = [
        np.arange(-90,91,2),
        np.zeros(np.arange(-90,91,2).shape)
    ]

    Llist = np.zeros(len(CraCaulist))
    SIDlist = np.ones(len(CraCaulist)) * 1191
    FOVlist = [400]*len(CraCaulist)
    imgSize = 512
    SOD = 820
    tdirList = [
        computeCameramWithPCangles(
            Pdeg,
            Cdeg,
            SOD,
            SID,
            FOV/imgSize,
            Ldeg
        ) for ((
            Ldeg,
            Pdeg,
            Cdeg,
            SID,
            FOV
        )) in zip(Llist, LaoRaolist, CraCaulist,SIDlist,FOVlist)
    ]
    v = generate_camera_vector(
        projection_center_iso, [voxel_size_iso] * 3,
        ct_isotropic.shape,tdirList,
        1,
        1
    )
    nz,ny,nx = ct_isotropic.shape
    vol_geom_ctiso = astra.create_vol_geom(ny, nx, nz)
    proj_geom_ctiso = astra.create_proj_geom('cone_vec',imgSize,imgSize,v)
    t1 = time.time()
    proj_id_ctiso, proj_data_ctiso = astra.create_sino3d_gpu(
            ct_isotropic, proj_geom_ctiso, vol_geom_ctiso
    )
    #proj_data_ctiso*=vox_sizes_mm[0]/vox_cbct_mm
    t2 = time.time()
    print(f"time for generating {proj_data_ctiso.shape[1]} views = {t1-t1:.2f}s")
    
    projImg = proj_data_ctiso.transpose(1,0,2)
    I0 = 1
    mu = 1.837e-2 # (mm-1) for water at 80kV
    projImg1 = I0*np.exp((-mu*voxel_size_iso/1000.0)*projImg)

    #rescaling . average of the center part of image at 512
    avgList = np.average(projImg1[:,250:-250,250:-250], axis=(1,2))
    projImg2 = np.asarray([proj*(512/avg) for proj, avg in zip(projImg1, avgList)])

    
    test = projImg2.astype(np.bool)
    projImg3 = projImg2.copy()
    #projImg2 = np.max(projImg2,axis=(1,2),keepdims=True) - projImg2
    projImg2 -= np.min(projImg)
    projImg2 /= np.max(projImg)
    #projImg2 = projImg * 255
    bin_img = 0.95 > projImg
    bin_img = 1 - bin_img
    return bin_img

In [ ]:
bin_img2 = generate_3d_dataset_artifact('../input/dataset-ct/dataset_ct/covid19/coronacases_org_001.nii')

In [ ]:
@interact(proj=fixed(bin_img2), index=(0, 64, 1))
def plot(proj, index):
    vmin = np.min(proj)
    vmax = np.max(proj)
    plt.imshow(proj[index], cmap='gray', vmin=vmin, vmax=vmax)

In [ ]:
!cd /kaggle/working
!mkdir liver

In [ ]:
import gc
from IPython.display import FileLink
gc.enable()

In [ ]:
os.chdir(r'/kaggle/working')

In [ ]:
gc.collect()

In [ ]:
n = 'liver'
path = '../input/dataset-ct/dataset_ct/' + n
for file in sorted(os.listdir(path)):
    filename  = path + '/' + file
    print(filename)
    proj = generate_3d_dataset_artifact(filename)
    np.savez_compressed('./' + n + '/'+file[:-4]+'.npz', PROJ=proj)
    del proj
    
    
    gc.collect()

In [ ]:
import zipfile

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

In [ ]:
zipf = zipfile.ZipFile(n+'.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('./'+n+'/', zipf)
zipf.close()

In [ ]:
FileLink(r'./'+n+'.zip')